参考: https://qiita.com/sugulu_Ogawa_ISID/items/697bd03499c1de9cf082

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import optuna
import torch.nn.functional as F
import torch.optim as optim

print(torch.__version__)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

1.11.0+cu113


device(type='cuda')

In [2]:
df = pd.DataFrame.from_dict(pd.read_pickle('./data/twitter/twitterJSA_data.pickle'))
lable2idx = {0:"ポジティブ&ネガティブ",
            1:"ポジティブ",
            2:"ネガティブ",
            3:"ニュートラル",
            4:"無関係"}
label_length = 5

# data数削減
df = df.sample(n=7000, random_state=123)
df.head

<bound method NDFrame.head of              id  topic              status            label  \
260782  1152146  10026  649586508110757888  [0, 0, 0, 0, 1]   
328485  1394459  10022  703752483483430912  [0, 0, 1, 0, 0]   
493161  2672788  10021  691170800846548992  [0, 0, 0, 1, 0]   
83628    478577  10000  668649960359878656  [0, 0, 1, 0, 0]   
57522    473962  10000  667307740990083073  [0, 0, 0, 1, 0]   
...         ...    ...                 ...              ...   
40864    457206  10000  661404228057849856  [0, 0, 0, 0, 1]   
258538  1154205  10024  460976864593326081  [0, 0, 0, 1, 0]   
15994     28434  10000  527276031233646592  [0, 0, 1, 1, 0]   
469170  2152918  10025  699154791004409856  [0, 0, 0, 1, 0]   
413335  2021103  10024  678222887090700288  [0, 0, 0, 1, 0]   

                                                     text  
260782                   コーテク垢までシャープに絡みに行ってるやんけｗｗｗｗｗｗｗｗｗｗ  
328485  パナ機今日も録画準備に入った所で電源落ちたよね\nJスポ4CC男子フリーの頭切れてるよね\n...  
493161            寒波により強気なので\nとりあ

In [3]:
train_df, test_df = train_test_split(df, test_size=0.3, shuffle=True)
train_df, valid_df = train_test_split(train_df, test_size=0.2, shuffle=True)
print(train_df["label"].value_counts())
print(test_df["label"].value_counts())

[0, 0, 0, 0, 1]    2019
[0, 0, 0, 1, 0]    1451
[0, 0, 1, 0, 0]     170
[0, 1, 0, 0, 0]      98
[0, 0, 0, 1, 1]      95
[0, 0, 1, 1, 0]      32
[0, 1, 0, 1, 0]      19
[0, 0, 1, 0, 1]      13
[0, 1, 0, 0, 1]      11
[1, 0, 0, 0, 0]       5
[1, 0, 0, 1, 0]       3
[1, 1, 0, 0, 0]       2
[1, 1, 1, 1, 1]       2
Name: label, dtype: int64
[0, 0, 0, 0, 1]    1095
[0, 0, 0, 1, 0]     765
[0, 0, 1, 0, 0]      93
[0, 1, 0, 0, 0]      49
[0, 0, 0, 1, 1]      49
[0, 0, 1, 1, 0]      16
[0, 0, 1, 0, 1]      10
[0, 1, 0, 1, 0]       8
[1, 0, 0, 0, 0]       5
[0, 1, 0, 0, 1]       5
[1, 0, 0, 1, 0]       2
[1, 1, 0, 0, 0]       2
[1, 0, 0, 0, 1]       1
Name: label, dtype: int64


In [4]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification
from transformers import ElectraTokenizer, ElectraForSequenceClassification, BertJapaneseTokenizer, BertForSequenceClassification, ElectraForPreTraining
# model_path = "cl-tohoku/bert-base-japanese"
model_path = "izumi-lab/electra-base-japanese-discriminator"
# model_path = "izumi-lab/electra-small-japanese-fin-discriminator"



In [5]:
tokenizer = BertJapaneseTokenizer.from_pretrained(model_path)
electra_model = ElectraForPreTraining.from_pretrained(model_path)
classifar_model = AutoModelForSequenceClassification.from_pretrained(model_path, problem_type="multi_label_classification", num_labels=label_length)

# freeze layers except last layer
for param in electra_model.parameters():
    param.requires_grad = False

'''
last_layer = list(electra_model.children())[-1]
print(f'except last layer: {last_layer}')
for param in last_layer.parameters():
    param.requires_grad = True
'''

Some weights of the model checkpoint at izumi-lab/electra-base-japanese-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at izumi-lab/electra-base-japanese-discriminator and are newly ini

"\nlast_layer = list(electra_model.children())[-1]\nprint(f'except last layer: {last_layer}')\nfor param in last_layer.parameters():\n    param.requires_grad = True\n"

In [6]:
max_len = 50
train_encoding = tokenizer(train_df["text"].to_list(), return_tensors="pt",padding=True, truncation=True, max_length=max_len)
valid_encoding = tokenizer(valid_df["text"].to_list(), return_tensors="pt",padding=True, truncation=True, max_length=max_len)
test_encoding = tokenizer(test_df["text"].to_list(), return_tensors="pt",padding=True, truncation=True, max_length=max_len)

In [7]:
train_label = torch.tensor(train_df["label"].to_list())
valid_label = torch.tensor(valid_df["label"].to_list())
test_label = torch.tensor(test_df["label"].to_list())

In [8]:
# 0 番目 の 5 トーク ン までの ID を 表示
print(train_encoding["input_ids"][ 0][: 50]) 
# 0 番目 の 5 トーク ン までの トー クン を デコード し て 表示
print(tokenizer.decode(train_encoding["input_ids"][0][:50]))

tensor([    1, 20109,    70, 20670, 13356, 12082,   700, 13951,   720,   767,
            2,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3])
[CLS] そもそも Xperia シリーズ が 好き だ わ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [9]:
class CreateDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__( self, idx):
        item = {key:torch.tensor(val[idx]).clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float32).clone().detach()
        return item
    
    def __len__( self):
        return len(self.labels)

train_dataset = CreateDataset(train_encoding, train_label)
val_dataset = CreateDataset(valid_encoding, valid_label)
test_dataset = CreateDataset(test_encoding, test_label)


In [10]:
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
validate_loader = DataLoader(val_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = True)

In [11]:
def saveModel(model): 
    path = "./IzumiElectraModel_multilabel_bceloss.pth" 
    torch.save(model.state_dict(), path)

In [12]:
def get_optimizer(trial, model):
  optimizer_names = ['Adam', 'MomentumSGD'] # 'rmsprop'
  optimizer_name = trial.suggest_categorical('optimizer', optimizer_names)
  
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-4)
  
  if optimizer_name == optimizer_names[0]: 
    adam_lr = trial.suggest_loguniform('adam_lr', 1e-4, 5e-3)
    optimizer = optim.Adam(model.parameters(), lr=adam_lr, weight_decay=weight_decay)
  elif optimizer_name == optimizer_names[1]:
    momentum_sgd_lr = trial.suggest_loguniform('momentum_sgd_lr', 1e-4, 5e-3)
    optimizer = optim.SGD(model.parameters(), lr=momentum_sgd_lr, momentum=0.9, weight_decay=weight_decay)
  else:
    optimizer = optim.RMSprop(model.parameters())
  
  return optimizer


def get_activation(trial):
    activation_names = ['ReLU', 'ELU', 'GELU']
    activation_name = trial.suggest_categorical('activation', activation_names)
    
    if activation_name == activation_names[0]:
      activation = F.relu
    elif activation_name == activation_names[1]:
      activation = F.elu
    else:
      activation = F.gelu
    return activation


class MLP(nn.Module):
    def __init__(self, trial):
      super().__init__()
      self.electra_model = electra_model
      self.activation_func = get_activation(trial) # relu elu gelu
      # 第1層
      self.fc1 = nn.Linear(768, 500)
      self.dropout1 = nn.Dropout(0.1)
      # 第2層
      self.fc2 = nn.Linear(500, 500)
      self.dropout2 = nn.Dropout(0.1)
      # 第3層
      self.fc3 = nn.Linear(500, 300)
      self.dropout3 = nn.Dropout(0.1)
      # 第4層
      self.fc4 = nn.Linear(300, label_length)
      
    def forward(self, x):
      x = self.activation_func(self.fc1(x))
      x = self.dropout1(x)
      x = self.activation_func(self.fc2(x))
      x = self.dropout2(x)
      x = self.activation_func(self.fc3(x))
      x = self.dropout3(x)
      output = self.activation_func(self.fc4(x))
      return output

In [60]:
sigmoid_func = torch.nn.Sigmoid()
loss_func = torch.nn.BCELoss()
EVAL_FUNC = True # if using "Exact Match" then "True", else using "Parts Match" then "False"

def train(model, device, train_loader, optimizer):
    model.train()
    running_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = electra_model(input_ids, attention_mask = attention_mask, output_hidden_states=True)
        last_hidden_states = outputs.hidden_states[-1][:,0,:]
        logits = model(last_hidden_states)
        loss = loss_func(sigmoid_func(logits), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)
        
def test(model, device, test_loader):
    model.eval()
    with torch.no_grad():
        acc_list = []
        losses_list = []
        running_loss = 0
        total = 0
        probs_list = []
        labels_list = []

        for batch in tqdm(test_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']
            outputs = electra_model(input_ids, attention_mask = attention_mask, output_hidden_states=True)
            last_hidden_states = outputs.hidden_states[-1][:,0,:]
            logits = model(last_hidden_states)
            test_loss = loss_func(sigmoid_func(logits).to('cpu'), labels)
            
            probs = sigmoid_func(logits).to('cpu')
            probs_list.append(probs)
            labels_list.append(labels.to('cpu'))

            # The label with the highest value will be our prediction 
            #_, predicted = torch.max(logits, 1) 

            total += logits.size(0) * logits.size(1)
            running_loss += test_loss.item()
 
        probs_ = torch.cat(probs_list, 0)
        labels_ = torch.cat(labels_list, 0)
        for th in range(1, 10, 1):
            correct = 0
            predicted = torch.where(probs_ > th/10, torch.ones(len(labels_), label_length), torch.zeros(len(labels_), label_length))
            if EVAL_FUNC:
                correct += (predicted == predicted).all(axis=1).sum().item()
                acc_list.append(correct / len(labels_))
            else:
                correct += (predicted == labels_).sum().item()
                acc_list.append(correct / total)

    max_acc = max(acc_list)
    max_acc_index = acc_list.index(max_acc)


    print(f"max_acc : {max_acc} then th=0.{max_acc_index+1}")
    return running_loss/len(test_loader), acc_list

In [61]:
EPOCH = 20
def objective(trial):
  device = "cuda" if torch.cuda.is_available() else "cpu"

  mlp_model= MLP(trial).to(device)
  optimizer = get_optimizer(trial, mlp_model)

  epoc_profile = {}
  max_acc = 0
  for step in range(EPOCH):
    train_loss = train(mlp_model, device, train_loader, optimizer)
    test_loss, acc_list = test(mlp_model, device, test_loader)

    epoc_profile[f"epoc{step}_test_loss"] = test_loss
    epoc_profile[f"epoc{step}_acc"] = acc_list
    epoc_profile[f"epoc{step}_train_loss"] = train_loss

    trial.set_user_attr(f'profile', epoc_profile)
    if max(acc_list) > max_acc:
      torch.save(mlp_model.state_dict(), f"./models/checkpoints/trialnum_{trial.number+1}.pth")
      max_acc = max(acc_list)
  return (1 - max(acc_list))
    

In [62]:
TRIAL_SIZE = 2
study = optuna.create_study()
study.optimize(objective, n_trials=TRIAL_SIZE)

[I 2022-08-25 12:51:47,338] A new study created in memory with name: no-name-b7449556-f368-46be-a6bc-044423523db7
  0%|          | 0/62 [00:00<?, ?it/s]C:\Users\windo\AppData\Local\Temp\ipykernel_32360\825136160.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]).clone().detach() for key, val in self.encodings.items()}
C:\Users\windo\AppData\Local\Temp\ipykernel_32360\825136160.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float32).clone().detach()
100%|██████████| 33/33 [00:04<00:00,  6.95it/s]


max_acc : 1.0 then th=0.1


 55%|█████▍    | 34/62 [00:06<00:05,  5.15it/s]


KeyboardInterrupt: 

In [54]:
study.best_params

{'activation': 'ELU',
 'optimizer': 'Adam',
 'weight_decay': 1.0114777524504494e-05,
 'adam_lr': 0.0012947367272763107}

In [55]:
study.best_trial

FrozenTrial(number=0, values=[0.47619047619047616], datetime_start=datetime.datetime(2022, 8, 25, 11, 58, 35, 34141), datetime_complete=datetime.datetime(2022, 8, 25, 12, 5, 15, 644978), params={'activation': 'ELU', 'optimizer': 'Adam', 'weight_decay': 1.0114777524504494e-05, 'adam_lr': 0.0012947367272763107}, distributions={'activation': CategoricalDistribution(choices=('ReLU', 'ELU', 'GELU')), 'optimizer': CategoricalDistribution(choices=('Adam', 'MomentumSGD')), 'weight_decay': LogUniformDistribution(high=0.0001, low=1e-06), 'adam_lr': LogUniformDistribution(high=0.005, low=0.0001)}, user_attrs={'profile': {'epoc0_test_loss': 0.49892083442572394, 'epoc0_acc': [0.0, 0.0, 0.023333333333333334, 0.36428571428571427, 0.36428571428571427, 0.0, 0.0, 0.0, 0.0], 'epoc0_train_loss': 0.5171533077955246, 'epoc1_test_loss': 0.47721914450327557, 'epoc1_acc': [0.0, 0.0, 0.023333333333333334, 0.45714285714285713, 0.42428571428571427, 0.0, 0.0, 0.0, 0.0], 'epoc1_train_loss': 0.48781480327729254, 'ep

In [56]:
hist_df = study.trials_dataframe(multi_index=True)
hist_df.to_pickle("log/profile.pickle")

In [57]:
hist_df

number     value             datetime_start          datetime_complete  \
                                                                           
0      0  0.476190 2022-08-25 11:58:35.034141 2022-08-25 12:05:15.644978   
1      1  0.478571 2022-08-25 12:05:15.645975 2022-08-25 12:11:59.842367   

                duration     params                                         \
                         activation   adam_lr momentum_sgd_lr    optimizer   
0 0 days 00:06:40.610837        ELU  0.001295             NaN         Adam   
1 0 days 00:06:44.196392       GELU       NaN         0.00076  MomentumSGD   

                                                       user_attrs     state  
  weight_decay                                            profile            
0     0.000010  {'epoc0_test_loss': 0.49892083442572394, 'epoc...  COMPLETE  
1     0.000067  {'epoc0_test_loss': 0.686696758776, 'epoc0_acc...  COMPLETE

In [58]:
pro = hist_df[('user_attrs','profile')].to_list()[1]
pro['epoc0_test_loss']

0.686696758776

In [59]:
pro.keys()

dict_keys(['epoc0_test_loss', 'epoc0_acc', 'epoc0_train_loss', 'epoc1_test_loss', 'epoc1_acc', 'epoc1_train_loss', 'epoc2_test_loss', 'epoc2_acc', 'epoc2_train_loss', 'epoc3_test_loss', 'epoc3_acc', 'epoc3_train_loss', 'epoc4_test_loss', 'epoc4_acc', 'epoc4_train_loss', 'epoc5_test_loss', 'epoc5_acc', 'epoc5_train_loss', 'epoc6_test_loss', 'epoc6_acc', 'epoc6_train_loss', 'epoc7_test_loss', 'epoc7_acc', 'epoc7_train_loss', 'epoc8_test_loss', 'epoc8_acc', 'epoc8_train_loss', 'epoc9_test_loss', 'epoc9_acc', 'epoc9_train_loss', 'epoc10_test_loss', 'epoc10_acc', 'epoc10_train_loss', 'epoc11_test_loss', 'epoc11_acc', 'epoc11_train_loss', 'epoc12_test_loss', 'epoc12_acc', 'epoc12_train_loss', 'epoc13_test_loss', 'epoc13_acc', 'epoc13_train_loss', 'epoc14_test_loss', 'epoc14_acc', 'epoc14_train_loss', 'epoc15_test_loss', 'epoc15_acc', 'epoc15_train_loss', 'epoc16_test_loss', 'epoc16_acc', 'epoc16_train_loss', 'epoc17_test_loss', 'epoc17_acc', 'epoc17_train_loss', 'epoc18_test_loss', 'epoc18_a

In [36]:
pro['epoc8_train_loss']

0.0